<a href="https://colab.research.google.com/github/OhadRubin/audio/blob/master/fine_tune_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tune Whisper For Werewolf

<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/whisper_architecture.svg" alt="Trulli" style="width:100%">

## Prepare Environment

In [ ]:
# !pip install --upgrade --quiet pip
# !pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

In [24]:
!pip install --quiet datasets evaluate jiwer

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

# Prepare Data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


  0%|          | 0/901 [00:00<?, ?it/s]

  0%|          | 0/896 [00:00<?, ?it/s]

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

import datasets
from tqdm.auto import tqdm


SAMPLING_RATE = 16000
MAX_DURATION = 30
import numpy as np
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from datasets import load_dataset, Audio, DatasetDict
from transformers import WhisperForConditionalGeneration


feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small")
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

werewolf_data = DatasetDict()

werewolf_data["train"] = load_dataset("parquet", data_files=["https://huggingface.co/datasets/iohadrubin/werewolf_dialogue_data_10sec/resolve/main/data/train-00002-of-00014-e0e6b0000eedceb4.parquet"])["train"]
werewolf_data["test"] = load_dataset("parquet", data_files=["https://huggingface.co/datasets/iohadrubin/werewolf_dialogue_data_10sec/resolve/main/data/train-00009-of-00014-603088eeb6352a9b.parquet"])["train"]

# werewolf_data = werewolf_data.cast_column("audio", Audio(sampling_rate=SAMPLING_RATE))
werewolf_data = werewolf_data.filter(lambda x: x["end"] - x["start"] <= MAX_DURATION and x["dialogue"][-1]["target"] is not None)

prompt_prefix = """Given the following dialogue and audio, assign the last utterance one or more of the following tags (delimited by commas):

'Accusation', 'Defense', 'Evidence', 'Identity Declaration', 'Interrogation', 'No Strategy'

```
"""

prompt_suffix = """
```

Reminder: Assign one or more of the following tags (delimited by commas): 'Accusation', 'Defense', 'Evidence', 'Identity Declaration', 'Interrogation', 'No Strategy'.

Assignment:
"""

def get_prompt(batch):
  dialogue = "\n".join(f"{x['speaker']}: {x['utterance']}" for x in batch['dialogue'])
  prompt = prompt_prefix + dialogue + prompt_suffix
  return prompt



def prepare_dataset(batch):

  prompt = get_prompt(batch)
  target = batch["dialogue"][-1]["target"]
  sentence = prompt + target

  input_ids = tokenizer.encode(sentence)
  if len(input_ids) > 446:
    input_ids = input_ids[:prompt_prefix_len] + input_ids[-max_prompt_suffix_len:]
  batch["decoder_input_ids"] = np.array(input_ids)

  labels = np.array(input_ids)
  target_len = len(tokenizer.encode(target)[2:])
  labels[:-target_len] = -100
  batch["labels"] = labels

  return batch


def add_audio(batch):
  batch["input_features"] = feature_extractor([x["array"] for x in batch["audio"]], sampling_rate=16000,return_tensors="pt").input_features
  return batch


werewolf_data["train"] = datasets.Dataset.from_list(list(tqdm(werewolf_data["train"])))
werewolf_data["test"] = datasets.Dataset.from_list(list(tqdm(werewolf_data["test"])))
prompt_prefix_len = len(tokenizer.encode(prompt_prefix)[:-1])
max_prompt_suffix_len = 446 - prompt_prefix_len
werewolf_data = werewolf_data.map(prepare_dataset)
werewolf_data = werewolf_data.map(add_audio, batched=True, batch_size=50)




from transformers import Seq2SeqTrainingArguments

import evaluate


metric = evaluate.load("wer")
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}



training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-werewolf",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    eval_strategy="steps",
    push_to_hub=True,
)
from transformers import Seq2SeqTrainer




  0%|          | 0/901 [00:00<?, ?it/s]

  0%|          | 0/896 [00:00<?, ?it/s]

Map:   0%|          | 0/901 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1048 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/896 [00:00<?, ? examples/s]

In [6]:
model = model.to("cuda")

In [13]:
len(werewolf_data["train"][0]["labels"])

448

In [16]:



@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
      input_features = [{"input_features": feature["input_features"]} for feature in features]
      batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")


      labels_batch = self.processor.tokenizer.pad([{"input_ids": feature["labels"]} for feature in features], return_tensors="pt")
      labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

      input_ids_batch = self.processor.tokenizer.pad([{"input_ids": feature["decoder_input_ids"]} for feature in features], return_tensors="pt")
      input_ids = input_ids_batch["input_ids"].masked_fill(input_ids_batch.attention_mask.ne(1), -100)

      if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
          labels = labels[:, 1:]

      batch["labels"] = labels
      batch["decoder_input_ids"] = input_ids

      return batch
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=werewolf_data["train"],
    eval_dataset=werewolf_data["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    # device="cuda"
)
trainer.train()

<ipython-input-16-5f658e775d2d>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### Training

Training will take approximately 5-10 hours depending on your GPU or the one
allocated to this Google Colab. If using this Google Colab directly to
fine-tune a Whisper model, you should make sure that training isn't
interrupted due to inactivity. A simple workaround to prevent this is
to paste the following code into the console of this tab (_right mouse click_
-> _inspect_ -> _Console tab_ -> _insert code_).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);
```

The peak GPU memory for the given training configuration is approximately 15.8GB.
Depending on the GPU allocated to the Google Colab, it is possible that you will encounter a CUDA `"out-of-memory"` error when you launch training.
In this case, you can reduce the `per_device_train_batch_size` incrementally by factors of 2
and employ [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps)
to compensate.

To launch training, simply execute:

KeyError: 'audio'

Our best WER is 32.0% - not bad for 8h of training data! We can make our model more accessible on the Hub with appropriate tags and README information.
You can change these values to match your dataset, language and model
name accordingly:

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: hi, split: test",
    "language": "hi",
    "model_name": "Whisper Small Hi - Sanchit Gandhi",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

The training results can now be uploaded to the Hub. To do so, execute the `push_to_hub` command and save the preprocessor object we created:

In [ ]:
trainer.push_to_hub(**kwargs)

## Building a Demo

Now that we've fine-tuned our model we can build a demo to show
off its ASR capabilities! We'll make use of 🤗 Transformers
`pipeline`, which will take care of the entire ASR pipeline,
right from pre-processing the audio inputs to decoding the
model predictions.

Running the example below will generate a Gradio demo where we
can record speech through the microphone of our computer and input it to
our fine-tuned Whisper model to transcribe the corresponding text:

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="sanchit-gandhi/whisper-small-hi")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()